In [ ]:
library(Seurat)
library(dplyr)
library(patchwork)
library(RColorBrewer)
library(wesanderson)
library(ggplot2)

In [ ]:
nuclei <- readRDS("full_nuclei.rds")
#islolate MSNs
isolatedMSNs <- subset(nuclei, idents = 'MSNs')
#get rid of poor quality cells
msns <- subset(isolatedMSNs, subset = nFeature_RNA > 200 & nFeature_RNA > 2500)

In [ ]:
#redo PCA
msns <- FindVariableFeatures(object = msns)
msns <- RunPCA(msns, features = VariableFeatures(object = msns))

In [ ]:
#redo umap
msns <- FindNeighbors(msns, dims = 1:15)
msns <- RunUMAP(msns, dims = 1:15)

In [ ]:
# the clustering parameter was optimized based on seperation of NUDAP from D1H
msns <- FindClusters(msns, resolution = 0.3)
head(Idents(msns), 5)

In [ ]:
#ClusterPlot
DimPlot(msns, reduction = "umap", label = TRUE, repel = TRUE)

In [ ]:
#We are not sure of the identity of this small cluster
msns <- SubsetData(object = msns, ident.remove = "11")

In [ ]:
#AnimalPlot
DimPlot(msns, reduction = "umap", group.by = "monkey")

In [ ]:
#RegionPlot
DimPlot(msns, reduction = "umap", group.by = "region_name")

In [ ]:
#ClusterPlot
DimPlot(msns, reduction = "umap", label = TRUE, repel = TRUE)

In [ ]:
Idents(object = msns) <- 'integrated_snn_res.0.3'
msns <- RenameIdents(object = msns,'0' = 'D1-Matrix','1' = 'D2-Matrix','2' = 'D2-Matrix','3' = 'D1-Matrix', '6' = 'D1-Striosome','7' = 'D2-Striosome', '4' = 'D1-Shell/OT','5' = 'D2-Shell/OT','8' = 'D1-NUDAP','9' = 'D1-ICj','10' = 'D1/D2-Hybrid')
msns@meta.data$MSN_type = Idents(object = msns)

In [ ]:
#ClusterPlot
DimPlot(msns,reduction = "umap",group.by = "MSN_type", label = TRUE, repel = TRUE)

In [ ]:
DimPlot(msns,reduction = "umap", cols = brewer.pal(10,"Paired"))

In [ ]:
DimPlot(msns, group.by = "region_name", cols = brewer.pal(3,"Accent"))

In [ ]:
FeaturePlot(msns, features = c("DRD2"))+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
FeaturePlot(msns, features = c("DRD1"))+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
FeaturePlot(msns, features = c("CASZ1"))+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
FeaturePlot(msns, features = c("DRD1", "TAC1","DRD2","PENK","RXFP1", "CPNE4"), blend = FALSE)+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
FeaturePlot(msns, features = c("FOXP2", "SEMA5B","TSHZ1", "OPRM1", "KHDRBS3","EPHA4"), blend = FALSE)+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
FeaturePlot(msns, features = c("KHDRBS2","GDA","KCNIP1","BACH2", "KHDRBS3","FAM163A","STXBP6"), blend = FALSE)+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
FeaturePlot(msns, features = c("GRIA4","GREB1L","ARHGAP6"), blend = FALSE)+scale_fill_gradientn(colors = brewer.pal(5,"Blues")) 

In [ ]:
Idents(object = msns) <- 'MSN_type'
my_levels <- c('D2-Matrix','D1-Matrix','D2-Striosome','D1-Striosome','D2-Shell/OT','D1-Shell/OT','D1-ICj','D1-NUDAP','D1/D2-Hybrid')
Idents(msns) <- factor(Idents(msns), levels= my_levels)

msns.markers <- FindAllMarkers(msns, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
msns.markers %>% group_by(cluster) %>% top_n(n = 2, wt = avg_logFC)
top10 <- msns.markers %>% group_by(cluster) %>% top_n(n = 10, wt = avg_logFC)
#DoHeatmap(msns, features = top10$gene, disp.min = -2, disp.max = 2, group.colors = brewer.pal(10,"Paired")) +scale_fill_gradientn(colors = brewer.pal(9,"PuOr"))



In [ ]:
unique(msns$MSN_type)

In [ ]:
Idents(object = msns) <- 'MSN_type'
my_levels <- c('D2-Matrix','D1-Matrix','D2-Striosome','D1-Striosome','D2-Shell/OT','D1-Shell/OT','D1-ICj','D1-NUDAP','D1/D2-Hybrid')
Idents(msns) <- factor(Idents(msns), levels= my_levels)

In [ ]:
DoHeatmap(subset(msns, downsample = 200),features = top10$gene, disp.min = -2, disp.max = 2, group.colors = brewer.pal(10,"Paired")) +scale_fill_gradientn(colors = brewer.pal(9,"PuOr"))